In [1]:
% matplotlib inline

from __future__ import division
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import acf, ccf
from statsmodels.tsa.tsatools import detrend
from scipy.spatial.distance import pdist, squareform
import scipy.signal as signal
import time as tm
import gsw as sw
from scipy.optimize import curve_fit
from scipy.stats import chi2

from mpl_toolkits.basemap import Basemap, cm
from netCDF4 import Dataset as NetCDFFile

In [2]:
def convert_KM(sds):
    lon_tmp = sds[(sds['cruise']=='KiloMoana_1')]['lon']
    lon_tmp = lon_tmp//100 + (lon_tmp - lon_tmp//100*100)/60
    lat_tmp = sds[(sds['cruise']=='KiloMoana_1')]['lat']
    lat_tmp = lat_tmp//100 + (lat_tmp - lat_tmp//100*100)/60
    
    sds.loc[sds['cruise']=='KiloMoana_1','lon'] = -lon_tmp
    sds.loc[sds['cruise']=='KiloMoana_1','lat'] = lat_tmp
    
    lon_tmp2 = sds[(sds['cruise']=='SCOPE_2')]['lon']
    lon_tmp2 = lon_tmp2//100 + (lon_tmp2 - lon_tmp2//100*100)/60
    lat_tmp2 = sds[(sds['cruise']=='SCOPE_2')]['lat']
    lat_tmp2 = lat_tmp2//100 + (lat_tmp2 - lat_tmp2//100*100)/60
    
    sds.loc[sds['cruise']=='SCOPE_2','lon'] = -lon_tmp2
    sds.loc[sds['cruise']=='SCOPE_2','lat'] = lat_tmp2
    
    return sds

def checklon(data):
    cxx = (data['lon']).values # longitude

    # convert all longitude to longitude east
    cxx[cxx<0]= 360+cxx[cxx<0];
    data['lon_e'] = cxx
    cxbew = cxx
    cxbew[cxbew>180] = cxbew[cxbew>180]-360   
    data['lon'] = cxbew
    return data

def distance(data):
    cruise = pd.unique(data['cruise'])
    distances = pd.DataFrame()    
    for c in cruise:
        temp = pd.DataFrame()
        temp = data.loc[(data['cruise']==c), ['cruise','file_time','lon','lat']]
        
        # calculate distance in km between points
        temp['dx'] = np.insert(sw.distance(temp['lon'].values, temp['lat'].values, 0),0,0)/1000
        temp['x'] = np.cumsum(temp['dx'])
        
        if c == cruise[0]:
            distances = distances.append(temp)
        else:
            distances = pd.concat([distances, temp], axis=0)
        del temp
    data = pd.merge(left = data, right = distances, how='left', left_on=['cruise','file_time','lat','lon'], right_on=['cruise','file_time','lat','lon'])
    #return distances

    return data  

def tracks(data):
    seg = []
    start = 0
    n = 1
    for i in range(len(data)):
        if ((data['dx'][i] > 20) | (data['dx'][i] < 0.01)):
            seg.append([data['cruise'][i], start, i-1, sum(data['dx'][start:i-1])])# - data['x'][start])])
                                                         
            n += 1
            start = i+1
    
    segments = pd.DataFrame(seg, columns = ['cruise', 'start', 'end', 'length']) 
    segments = segments.loc[(segments['length'] >= 200)]
    segments = segments.loc[(segments['end'] > 0)]
    segments['number'] = range(1, len(segments)+1)
    segments = segments.set_index(['number'])
    return segments


def interpdata(track, aa, bb, var):
    # we went to create a new dataframe which has the interpolated data for each track for all variables, 
    # and a position.
    intdata = pd.DataFrame()
    # set the interpolation distance 
    n = 1 # 1km 
    begin = np.floor(track['x'][:1])
    intdata['x'] = np.arange(aa, bb, n)
 
    for v in var:
        intdata[v] = np.interp(intdata['x'].values,track['x'].values,track[v].values)
          
    return intdata


In [3]:
# import data
filePath = '/Users/archie/Google Drive/seaflow_data/KM1_abun_sds_opp.csv'
data = pd.read_csv(filePath, sep=',')
print data.columns

filePath = r'/Users/archie/Google Drive/seaflow_data/sds_depth.csv'
depth = pd.read_csv(filePath)

filePath = r'/Users/archie/Google Drive/seaflow_data/sds_dist2coast.csv'
dcoast = pd.read_csv(filePath)


Index([u'Unnamed: 0', u'cruise', u'file_time', u'opp_evt_ratio', u'flow_rate',
       u'beads', u'picoeuk', u'prochloro', u'synecho', u'unknown', u'cocco',
       u'ts', u'file_duration', u'pico', u'Unnamed: 0.1', u'tot_chl',
       u'file_time_r', u'Unnamed: 0.1', u'file_duration_r', u'lat', u'lon',
       u'conductivity', u'salinity', u'ocean_tmp', u'par', u'bulk_red',
       u'stream_pressure', u'flow_rate_r', u'event_rate', u'avg_chl',
       u'avg_pe', u'avg_fsc', u'density', u'lon_e', u'dx', u'x', u'sds_time',
       u'file_time_r.1', u'opp.evt'],
      dtype='object')


In [4]:
# add density 
data['density'] = sw.rho(data['salinity'].values,data['ocean_tmp'].values,0)
data = checklon(data)

# only keep KM1304 data
data = data[(data['cruise']=='KiloMoana_1')]
               
# sort the dataframe so that it is in date/time order
data = pd.merge(data, depth, on=['cruise','file_time'], how='inner')
data = pd.merge(data, dcoast, on=['cruise','file_time'], how='inner')
data = data.sort_values(by = ['cruise', 'file_time'],ascending=[1, 1])
data = data.reset_index(drop=True)

# now we have a dataframe with the total alongtrack distance for each cruise, 
# and the distance between points for each file.

clean = data[['cruise', 'file_time', 'lon', 'lon_e', 'lat', 'depth', 'dcoast', 'salinity', 'ocean_tmp', 'density', 'pico', 'synecho', 'picoeuk']]
clean['tot_chl'] = data['tot_chl']#/(data['file_duration']*data['opp.evt']*data['flow_rate'])
clean = clean.sort_values(by = ['cruise', 'file_time'],ascending=[1, 1])
clean = distance(clean)


/Users/archie/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [25]:
# look for correlations between variables over the entire cruise track
clean.corr()



,lon,lon_e,lat,depth,dcoast,salinity,ocean_tmp,density,pico,synecho,picoeuk,tot_chl,dx,x
lon,1.000000,1.000000,0.751694,0.842161,-0.564667,-0.792679,-0.759462,0.317966,-0.488513,0.536684,0.170639,0.032492,-0.065404,-0.926388
lon_e,1.000000,1.000000,0.751694,0.842161,-0.564667,-0.792679,-0.759462,0.317966,-0.488513,0.536684,0.170639,0.032492,-0.065404,-0.926388
lat,0.751694,0.751694,1.000000,0.537517,-0.192628,-0.942439,-0.963168,0.669741,-0.798800,0.451297,0.615968,-0.185894,-0.042456,-0.944212
depth,0.842161,0.842161,0.537517,1.000000,-0.744561,-0.691840,-0.668196,0.221794,-0.293634,0.396922,0.074368,-0.039488,-0.047895,-0.732526
dcoast,-0.564667,-0.564667,-0.192628,-0.744561,1.000000,0.250082,0.324910,-0.277072,0.283556,-0.321973,-0.117261,-0.236237,0.025294,0.400672
salinity,-0.792679,-0.792679,-0.942439,-0.691840,0.250082,1.000000,0.963630,-0.479169,0.654562,-0.452701,-0.452788,0.234177,0.050486,0.933737
ocean_tmp,-0.759462,-0.759462,-0.963168,-0.668196,0.324910,0.963630,1.000000,-0.689563,0.748681,-0.430365,-0.599616,0.268686,0.039679,0.931344
density,0.317966,0.317966,0.669741,0.221794,-0.277072,-0.479169,-0.689563,1.000000,-0.776444,0.230273,0.794053,-0.214087,0.008027,-0.546516
pico,-0.488513,-0.488513,-0.798800,-0.293634,0.283556,0.654562,0.748681,-0.776444,1.000000,-0.527416,-0.698366,0.014969,0.000586,0.700088
synecho,0.536684,0.536684,0.451297,0.396922,-0.321973,-0.452701,-0.430365,0.230273,-0.527416,1.000000,0.120492,0.358169,-0.057033,-0.525376


In [33]:
# look at correlations for each segment individually
segments = tracks(clean)


for k in range(1, len(segments)+1):
    a = segments['start'][k]
    b = segments['end'][k]
        
    tmp = data[['density', 'ocean_tmp', 'salinity', 'synecho', 'pico', 'picoeuk', 'tot_chl']][a:b]
    
    pval = np.zeros([tmp.shape[1],tmp.shape[1]])
    for i in range(tmp.shape[1]): # rows are the number of rows in the matrix.
        for j in range(tmp.shape[1]):
            JonI        = pd.ols(y=tmp.icol(i), x=tmp.icol(j), intercept=True)
            pval[i,j]  = np.round(JonI.f_stat['p-value'],8)

    
    print 'segment', k
    print tmp.corr()
    print pval
    del tmp, pval

segment 1
            density  ocean_tmp  salinity   synecho  pico   picoeuk   tot_chl
density    1.000000  -0.602600  0.958690 -0.810807   NaN  0.539534 -0.694484
ocean_tmp -0.602600   1.000000 -0.350701  0.213386   NaN -0.018100  0.324414
salinity   0.958690  -0.350701  1.000000 -0.875508   NaN  0.626676 -0.699463
synecho   -0.810807   0.213386 -0.875508  1.000000   NaN -0.487898  0.813328
pico            NaN        NaN       NaN       NaN   NaN       NaN       NaN
picoeuk    0.539534  -0.018100  0.626676 -0.487898   NaN  1.000000 -0.181457
tot_chl   -0.694484   0.324414 -0.699463  0.813328   NaN -0.181457  1.000000
[[  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
               nan   0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   1.00000000e-08   5.31920000e-04
               nan   7.71459300e-01   9.00000000e-08]
 [  0.00000000e+00   1.00000000e-08   0.00000000e+00   0.00000000e+00
               nan   0.00000000e+00   0.00000000e+00]
 [

/Users/archie/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:14: FutureWarning: icol(i) is deprecated. Please use .iloc[:,i]
